In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
% cd gdrive/My\ Drive/kaggle/digit

In [ ]:
from google.colab import files
files.upload()  #this will prompt you to upload the kaggle.json

###### dataset is available on https://www.kaggle.com/c/digit-recognizer/data

In [ ]:
import pandas as pd
import cv2
from PIL import Image
from keras.utils.np_utils import to_categorical
from matplotlib import pyplot as plt
import numpy as np
import xgboost
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Convolution2D, MaxPooling2D, Flatten, Input, Dropout
from keras.optimizers import Adam
%matplotlib inline

In [ ]:
df=pd.read_csv('train.csv')
dfT=pd.read_csv('test.csv')
dfS=pd.read_csv('sample_submission.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
labelImages=df['label'].values

In [ ]:
Y_train =to_categorical(df['label'].values, num_classes=10)

In [ ]:
df.drop('label',axis=1,inplace=True)
X_train=df

In [ ]:
trainImages=np.asarray([rows.values.reshape(28,28,1) for index,rows in df.iterrows()],dtype=float)

In [ ]:
X_tr, X_test, y_tr, y_test = train_test_split(X_train, labelImages, test_size=0.2, random_state=42)

In [ ]:
X_test=np.asarray([rows.values.reshape(28,28,1) for index,rows in X_test.iterrows()],dtype=float)
y_test = to_categorical(y_test, num_classes=10)

In [ ]:
inputs = Input(shape=(28,28,1))

conv2d1 = Convolution2D(filters=64, kernel_size=6, padding='same', data_format='channels_last')(inputs)
activation1 = Activation('relu')(conv2d1)
maxpooling1 = MaxPooling2D(pool_size=3, strides=3, padding='same', data_format='channels_last')(activation1)
dropout1 = Dropout(0.25)(maxpooling1)

conv2d2 = Convolution2D(filters=64, kernel_size=5, padding='same', data_format='channels_last')(dropout1)
activation2 = Activation('relu')(conv2d2)
maxpooling2 = MaxPooling2D(pool_size=3, strides=3, padding='same', data_format='channels_last')(activation2)

flatten = Flatten()(maxpooling2)

dense1 = Dense(1024)(flatten)
activation3 = Activation('relu')(dense1)

dense2 = Dense(10)(activation3)
activation4 = Activation('softmax')(dense2)

model_dense2_output = Model(inputs=inputs, outputs=activation3)
model = Model(inputs=inputs, outputs=activation4)

In [ ]:
adam = Adam(lr=1e-4)
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(trainImages, Y_train, epochs=2, batch_size=16)
print('\ntest loss: ', loss)
print('\ntest accuracy: ', accuracy)

In [ ]:
dfT=np.asarray([rows.values.reshape(28,28,1) for index,rows in dfT.iterrows()],dtype=float)

In [ ]:
y_pred=model.predict(dfT)

In [ ]:
Y=[]
for i in range(0,len(y_pred)):
  Y.append(tf.argmax(y_pred[i]).numpy())

In [ ]:
len(Y)

In [ ]:
dfS.drop('Label',axis=1,inplace=True)
dfS['Label']=Y
dfS.to_csv('submissionConv15epoch.csv',index=False)

## Score 0.98671 with CNN only

In [ ]:
X_train_xg = model_dense2_output.predict(trainImages)
# print(np.array(X_train_xg).shape)
# print(np.array(y_train).shape)

z = np.concatenate([np.array(X_train_xg).reshape(42000,1024),np.array(labelImages).reshape(42000,1)],axis=1)
z = pd.DataFrame(z)
z.to_csv('train_xg.csv',index=False)

train = pd.read_csv('train_xg.csv')
train_y = train['1024'].astype('int')
train_x = train.drop(['1024'],axis=1)
dataset = xgboost.DMatrix(train_x, label=train_y)
watchlist = [(dataset, 'train')]
params = {'max_depth':7, 'eta':0.1, 'silent':1, 'num_class':10,'objective':'multi:softmax' } 
model_xg = xgboost.train(params, dataset, num_boost_round=150, evals=watchlist)

In [ ]:
X_test_xg = model_dense2_output.predict(dfT)
z = np.array(X_test_xg).reshape(28000,1024)
z = pd.DataFrame(z)
z.to_csv('test_xg.csv',index=False)

In [ ]:
test = pd.read_csv('test_xg.csv')

In [ ]:
test= xgboost.DMatrix(test)
result = model_xg.predict(test)


In [ ]:
result=np.asanyarray(result,dtype=int)

In [ ]:
dfXgb=pd.read_csv('sample_submission.csv')
dfXgb.drop('Label',axis=1,inplace=True)
dfXgb['Label']=result
dfXgb.to_csv('submissionConv2dXgb.csv',index=False)

## Score 0.98685 with CNN+Xgboost